## Erkennung von Kreditkartenbetrug
***
##### von Romeo, Marko, Nikola und Miro

In [2]:
# Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as py

# seaborn graphics settings
sns.set(color_codes=True)

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pandas'

### Kontext 


#### Datenquelle

[kaggle](https://www.kaggle.com/mlg-ulb/creditcardfraud) - Das genannte Datenset beschäftigt sich mit der Erkennung von Kreditkartenbetrug und enthält verschiedene Informationen zu vergangenen Transaktionen. Es wurde von einer nicht genannten Institution bereitgestellt.

#### Beschaffung der Daten
Die genaue Beschaffungsquelle für das Datenset wurde nicht angegeben. Es wurde jedoch erwähnt, dass das Datenset von einer nicht genannten Institution bezogen wurde. 

#### Problem- und Fragestellung

Problemstellung:
1. Kreditkartenbetrug zu erkennen.

Fragestellung:
1. Wie können wir mithilfe eines Modells Kreditkartenbetrug von legitimen Transaktionen unterscheiden? 
2. Welche Merkmale und Muster in vergangenen Transaktionen können verwendet werden, um betrügerische Aktivitäten zu identifizieren?

#### Zielsetzung
Ziel ist es, ein Klassifizierungsmodell zu entwickeln, das auf Basis historischer Transaktionsdaten eine binäre Einschätzung für jede neue Kreditkartenbuchung gibt

#### Vorgehen & Methoden

##### Importing the data

In [2]:
df = pd.read_csv('card_transdata.csv', sep=',', encoding='utf-8')

# Get number of rows and columns
print(df.shape)

# Show first records
df.head(5)

(1000000, 8)


,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


##### Show data types

In [3]:
df.dtypes

distance_from_home                float64
distance_from_last_transaction    float64
ratio_to_median_purchase_price    float64
repeat_retailer                   float64
used_chip                         float64
used_pin_number                   float64
online_order                      float64
fraud                             float64
dtype: object

##### Show columns

In [4]:
df.describe()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000
mean,26.628792,5.036519,1.824182,0.881536,0.350399,0.100608,0.650552,0.087403
std,65.390784,25.843093,2.799589,0.323157,0.477095,0.300809,0.476796,0.282425
min,0.004874,0.000118,0.004399,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.878008,0.296671,0.475673,1.000000,0.000000,0.000000,0.000000,0.000000
50%,9.967760,0.998650,0.997717,1.000000,0.000000,0.000000,1.000000,0.000000
75%,25.743985,3.355748,2.096370,1.000000,1.000000,0.000000,1.000000,0.000000
max,10632.723672,11851.104565,267.802942,1.000000,1.000000,1.000000,1.000000,1.000000


##### correlation matrix

In [5]:
corr = df.corr()
corr

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
distance_from_home,1.000000,0.000193,-0.001374,0.143124,-0.000697,-0.001622,-0.001301,0.187571
distance_from_last_transaction,0.000193,1.000000,0.001013,-0.000928,0.002055,-0.000899,0.000141,0.091917
ratio_to_median_purchase_price,-0.001374,0.001013,1.000000,0.001374,0.000587,0.000942,-0.000330,0.462305
repeat_retailer,0.143124,-0.000928,0.001374,1.000000,-0.001345,-0.000417,-0.000532,-0.001357
used_chip,-0.000697,0.002055,0.000587,-0.001345,1.000000,-0.001393,-0.000219,-0.060975
used_pin_number,-0.001622,-0.000899,0.000942,-0.000417,-0.001393,1.000000,-0.000291,-0.100293
online_order,-0.001301,0.000141,-0.000330,-0.000532,-0.000219,-0.000291,1.000000,0.191973
fraud,0.187571,0.091917,0.462305,-0.001357,-0.060975,-0.100293,0.191973,1.000000


### Explorative Datenanalyse (EDA) (Niki)
Daten beschreiben, Identifizierung und Verstehen von Beziehungen und Gruppen,

### Hypothesentest (Marko)
Unifying & Transformation Datentypen und Formate analysieren und bei Bedarf vereinheitlichen, Daten sinnvoll zusammenführen

### Data Cleansing (Romeo)
Beurteilung der Datenqualität und erforderliche Bereinigung der Daten

### Analysis & Validation
- Analyse der Verteilungen, Abweichungen, Kennzahlen gemäss Unterricht

### Feature Selection & Engineering 
- Analyse der Korrelationen und Kausalitäten, Identifikation der Features (erklärende Variablen),
ggf. Optimierung der Datengrundlage 

### Anwendung in Machine Learning
- Entwicklung von Regressions- oder Klassifikationsmodellen (mind. 2 Modelle),
Performance-Vergleich und –Interpretation

### Anwendung von XAI – Techniken
- Anwendung von XAI – Techniken, um die entwickelten Modelle
in Bezug auf die Fallstudie zu erklären

##### Jupyter notebook --footer info--

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')